In [23]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.iolib.summary2 import summary_col

In [18]:
# Забираем датасет, предобработанный для проведения регрессионного анализа
!wget https://nc.cloud.cpur.ru/s/zAaYy5EDMySndAC/download/declarations_for_regression.xlsx

--2021-11-30 10:59:13--  https://nc.cloud.cpur.ru/s/zAaYy5EDMySndAC/download/declarations_for_regression.xlsx
Resolving nc.cloud.cpur.ru (nc.cloud.cpur.ru)... 84.201.157.40
Connecting to nc.cloud.cpur.ru (nc.cloud.cpur.ru)|84.201.157.40|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 774872 (757K) [application/vnd.openxmlformats-officedocument.spreadsheetml.sheet]
Saving to: ‘declarations_for_regression.xlsx’

declarations_for_re 100%[===================>] 756.71K  1.05MB/s    in 0.7s    

2021-11-30 10:59:15 (1.05 MB/s) - ‘declarations_for_regression.xlsx’ saved [774872/774872]



In [19]:
servants = pd.read_excel('declarations_for_regression.xlsx')
len(servants)

In [21]:
servants.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17223 entries, 0 to 17222
Data columns (total 9 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   Unnamed: 0                        17223 non-null  int64  
 1   name                              17223 non-null  object 
 2   position_standard                 17223 non-null  object 
 3   gender                            17223 non-null  object 
 4   married                           17223 non-null  object 
 5   year                              17223 non-null  int64  
 6   state_agency_short                17223 non-null  object 
 7   children                          17223 non-null  int64  
 8   income_diff_month_const_чиновник  17223 non-null  float64
dtypes: float64(1), int64(3), object(5)
memory usage: 1.2+ MB


In [25]:
# создаем переменную: наличине несоврешеннолетних детей
servants['if_children'] = np.where(servants['children'] == 0, 'no', 'yes')

In [26]:
#согарифмируем месячный доход чиноников
servants['log_income_diff'] = np.log(servants['income_diff_month_const_чиновник'])

In [28]:
# создаем категориальные переменнуые года и количества детей
servants['year_cat'] = servants['year'].astype(str)

## 1. Регрессия на данных всех министерств
Последовательно добавляем объясняющие переменные

In [37]:
# только пол
mdl_g = ols('log_income_diff ~ gender', data=servants).fit()
pprint(mdl_g.summary())

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
Dep. Variable:        log_income_diff   R-squared:                       0.015
Model:                            OLS   Adj. R-squared:                  0.015
Method:                 Least Squares   F-statistic:                     266.7
Date:                Tue, 30 Nov 2021   Prob (F-statistic):           1.65e-59
Time:                        11:13:43   Log-Likelihood:                -14913.
No. Observations:               17223   AIC:                         2.983e+04
Df Residuals:                   17221   BIC:                         2.985e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
In

In [38]:
# пол и семейный статус
mdl_g_m = ols('log_income_diff ~ gender + married', data=servants).fit()
pprint(mdl_g_m.summary())

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
Dep. Variable:        log_income_diff   R-squared:                       0.018
Model:                            OLS   Adj. R-squared:                  0.018
Method:                 Least Squares   F-statistic:                     162.1
Date:                Tue, 30 Nov 2021   Prob (F-statistic):           1.84e-70
Time:                        11:13:43   Log-Likelihood:                -14885.
No. Observations:               17223   AIC:                         2.978e+04
Df Residuals:                   17220   BIC:                         2.980e+04
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------

In [39]:
# Добавляем наличие детей
mdl_g_m_ch = ols('log_income_diff ~ gender + married + if_children', data=servants).fit()
pprint(mdl_g_m_ch.summary())

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
Dep. Variable:        log_income_diff   R-squared:                       0.020
Model:                            OLS   Adj. R-squared:                  0.019
Method:                 Least Squares   F-statistic:                     114.9
Date:                Tue, 30 Nov 2021   Prob (F-statistic):           1.07e-73
Time:                        11:13:44   Log-Likelihood:                -14875.
No. Observations:               17223   AIC:                         2.976e+04
Df Residuals:                   17219   BIC:                         2.979e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------

In [40]:
# добавляем министерство
mdl_g_m_ch_st = ols('log_income_diff ~ gender + married + if_children + state_agency_short', data=servants).fit()
pprint(mdl_g_m_ch_st.summary())

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
Dep. Variable:        log_income_diff   R-squared:                       0.095
Model:                            OLS   Adj. R-squared:                  0.094
Method:                 Least Squares   F-statistic:                     99.85
Date:                Tue, 30 Nov 2021   Prob (F-statistic):               0.00
Time:                        11:13:44   Log-Likelihood:                -14190.
No. Observations:               17223   AIC:                         2.842e+04
Df Residuals:                   17204   BIC:                         2.856e+04
Df Model:                          18                                         
Covariance Type:            nonrobust                                         
                                              coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------

In [41]:
# добавляем год
mdl_g_m_ch_st_y = ols('log_income_diff ~ gender + married + if_children + state_agency_short + year_cat', data=servants).fit()
pprint(mdl_g_m_ch_st_y.summary())

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
Dep. Variable:        log_income_diff   R-squared:                       0.113
Model:                            OLS   Adj. R-squared:                  0.112
Method:                 Least Squares   F-statistic:                     87.76
Date:                Tue, 30 Nov 2021   Prob (F-statistic):               0.00
Time:                        11:13:44   Log-Likelihood:                -14011.
No. Observations:               17223   AIC:                         2.807e+04
Df Residuals:                   17197   BIC:                         2.828e+04
Df Model:                          25                                         
Covariance Type:            nonrobust                                         
                                              coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------

In [42]:
# добавляем должность
mdl_g_m_ch_st_y_pos = ols('log_income_diff ~ gender + married + if_children + state_agency_short + year_cat + position_standard', data=servants).fit()
pprint(mdl_g_m_ch_st_y_pos.summary())

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
Dep. Variable:        log_income_diff   R-squared:                       0.513
Model:                            OLS   Adj. R-squared:                  0.512
Method:                 Least Squares   F-statistic:                     583.9
Date:                Tue, 30 Nov 2021   Prob (F-statistic):               0.00
Time:                        11:13:45   Log-Likelihood:                -8851.7
No. Observations:               17223   AIC:                         1.777e+04
Df Residuals:                   17191   BIC:                         1.802e+04
Df Model:                          31                                         
Covariance Type:            nonrobust                                         
                                                              coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------

Формируем общую таблицу с результатами регрессии на всех данных

In [43]:
res_table_all = summary_col([mdl_g, mdl_g_m, mdl_g_m_ch, mdl_g_m_ch_st, mdl_g_m_ch_st_y, mdl_g_m_ch_st_y_pos], 
                            stars=True,float_format='%0.3f', info_dict={'N':lambda x: "{0:d}".format(int(x.nobs)),
                             'R2':lambda x: "{:.3f}".format(x.rsquared)},
                             model_names=['(1)','(2)','(3)', '(4)', '(5)', '(6)'])

In [44]:
res_table_all.tables[0]

,(1),(2),(3),(4),(5),(6)
Intercept,12.133***,12.104***,12.096***,11.884***,11.954***,12.884***
,(0.006),(0.007),(0.008),(0.021),(0.025),(0.022)
gender[T.m],0.143***,0.121***,0.115***,0.138***,0.139***,0.001
,(0.009),(0.009),(0.009),(0.009),(0.009),(0.007)
if_children[T.yes],,,0.043***,0.054***,0.052***,-0.009
...,...,...,...,...,...,...
,,,,,(0.018),(0.013)
year_cat[T.2020],,,,,0.039**,0.004
,,,,,(0.018),(0.013)
N,17223,17223,17223,17223,17223,17223


## 2. Регрессия групп министерств

In [104]:
# функция для последовательного добавления объясняющих переменных
# и формирования общей таблицы с результатами

def regress(data):
  #Только пол
  mdl_g = ols('log_income_diff ~ gender', data=data).fit()
  #Добавляем семейный статус и наличие детей
  mdl_g_pers = ols('log_income_diff ~ gender + married + if_children', data=data).fit()
  #Добавляем министерство и год
  mdl_g_pers_st = ols('log_income_diff ~ gender + married + if_children + state_agency_short + year_cat', data=data).fit()
  #Добавляем должность
  mdl_full = ols('log_income_diff ~ gender + married + if_children + state_agency_short + year_cat + position_standard', data=data).fit()

  res_table = summary_col([mdl_g, mdl_g_pers, mdl_g_pers_st, mdl_full],
                          stars=True, float_format='%0.3f',
                           info_dict={'N':lambda x: "{0:d}".format(int(x.nobs)),
                                      'R2':lambda x: "{:.3f}".format(x.rsquared)},
                           model_names=['(1)','(2)','(3)', '(4)'])

  result = [res_table, mdl_full]
  return result



### **Группа 4:**

In [48]:
state_list = ['Минкультуры', 'Минстрой', 'Минтруд', 'Минздрав', 'Минсельхоз', 'Минпромторг']
group_4 = servants[servants['state_agency_short'].isin(state_list)]
len(group_4)

6142

**Регрессия без эффекта взаимодействия.
Последовательное добавление переменных**

In [ ]:
res_table_4, mdl_4_no_int_full = regress(group_4)
res_table_4.tables[0]

**Со взаимодействием**

In [ ]:
mdl_g4_full = ols('log_income_diff ~ position_standard + state_agency_short + married + gender + gender*married  + gender*if_children + year_cat', data=group_4).fit()

### **Группа 3:**

In [89]:
state_list = ['Минцифры', 'Минобр', 'Минюст']
group_3 = servants[servants['state_agency_short'].isin(state_list)]
len(group_3)

2711

**Без эффекта взаимодействия**

In [ ]:
res_table_3, mdl_3_no_int_full = regress(group_3)
res_table_3.tables[0]

**Со взаимодействием**

In [96]:
mdl_g3_full = ols('log_income_diff ~ position_standard + state_agency_short + married + gender + gender*married + gender*if_children + year_cat', data=group_3).fit()

### **Группа 1:**

In [71]:
state_list = ['Минэнерго', 'Минфин', 'Минтранс', 'Минприроды']
group_1 = servants[servants['state_agency_short'].isin(state_list)]
len(group_1)

4587

**Без эффекта взаимодействия**

In [ ]:
res_table_1, mdl_1_no_int_full = regress(group_1)
res_table_1.tables[0]

**Со взаимодействием**

In [79]:
mdl_g1_full = ols('log_income_diff ~ position_standard + state_agency_short + married + gender + gender*married + gender*if_children + year_cat', data=group_1).fit()

### **Группа 2:**

In [80]:
state_list = ['Минвостокразвития', 'Минэкономразвития', 'Минспорт']
group_2 = servants[servants['state_agency_short'].isin(state_list)]
len(group_2)

3783

**Без эффекта взаимодействия**

In [ ]:
res_table_2, mdl_2_no_int_full = regress(group_2)
res_table_2.tables[0]

Со взаимодействием

In [103]:
mdl_g2_full = ols('log_income_diff ~ position_standard + state_agency_short + gender + married + gender*married + gender*if_children + year_cat', data=group_2).fit()

**СВОДНАЯ ТАБЛИЦА ПО ВСЕМ ГРУПАМ БЕЗ ВЗАИМОДЕЙСТВИЯ**



In [109]:
res_table_groups_no_int = summary_col([mdl_1_no_int_full, mdl_2_no_int_full, mdl_3_no_int_full,
                                       mdl_4_no_int_full],stars=True, float_format='%0.3f',
                                       info_dict={'N':lambda x: "{0:d}".format(int(x.nobs)),
                             'R2':lambda x: "{:.3f}".format(x.rsquared)},
                              model_names=['Группа 1', 'Группа 2', 'Группа 3', 'Группа 4'])

In [ ]:
res_table_groups_no_int.tables[0]

**СВОДНАЯ ТАБЛИЦА ПО ВСЕМ ГРУПАМ СО ВЗАИМОДЕЙСТВИЕМ**

In [111]:
res_table_int = summary_col([mdl_g1_full, mdl_g2_full, mdl_g3_full, mdl_g4_full],
                            stars=True, float_format='%0.3f', info_dict={'N':lambda x: "{0:d}".format(int(x.nobs)),
                             'R2':lambda x: "{:.3f}".format(x.rsquared)}, model_names=['Группа 1', 'Группа 2', 'Группа 3', 'Группа 4'])

In [112]:
res_table_int.tables[0]

,Группа 1,Группа 2,Группа 3,Группа 4
Intercept,13.000***,12.959***,12.851***,12.914***
,(0.032),(0.047),(0.042),(0.026)
gender[T.m],-0.089***,-0.086***,0.068***,-0.044**
,(0.022),(0.022),(0.025),(0.018)
gender[T.m]:if_children[T.yes],0.006,-0.009,0.042,-0.046**
...,...,...,...,...
,(0.023),(0.039),(0.036),(0.021)
year_cat[T.2020],0.007,-0.200***,0.027,0.059***
,(0.024),(0.040),(0.035),(0.021)
N,4587,3783,2711,6142
